# **PNSR Criteria Calculation**

The goal in this code is to calculate the PNSR criteria for the generated dataset and the MPRNet pretrainted model as an example (calculation is the same for other networks we studied) to evaluate the performance of the networks. The PNSR calculation is done for random patches cropped from the image (with patch size "PS" chosen in the .YML file) similar to the calculation for MPRNet paper.

Finally, the PSNR of the best saved model from training is also compared to other values, suggesting that the fine-tuning improved the performance sinificantly.

**Note:** PNSR is not the optimization criterion when training the MPRNet, but it is the criteria that the training will decide to save the best model based on. PNSR is not differentiable, so Charbonnier loss is the main optimization criterion being used.

##########################################################################################

**1)** We are using the best configs of google colab to run all of our codes. Therefore, the first step is to mount google drive, where we have uploaded the MPRNet github codes + our datasets + pretrained and best trained models:

In [3]:
import os

#mount drive
%cd ..
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive

/
Mounted at /content/gdrive


**2)** Make the MPRNet directory, the main working directory:

In [4]:
os.chdir('mydrive/MPRNET_V2/Deblurring')
os.getcwd()
!ls

checkpoints   dataset_RGB.py	     losses.py		       training3.yml
checkpoints2  Datasets		     MPRNet.py		       training4.yml
CHK2	      Datasets2		     MPRNET_Training_V1.ipynb  training5.yml
CHK3	      Datasets3		     pretrained_models	       training6.yml
CHK4	      Datasets4		     __pycache__	       training7.yml
CHK5	      Datasets5		     README.md		       training_Nov.yml
CHK6	      evaluate_GOPRO_HIDE.m  Result		       training.yml
config.py     evaluate_RealBlur.py   test.py		       train.py
data_RGB.py   Input		     training2.yml	       utils


**3)** Install required packages:

In [5]:
!pip install yacs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!pip install warmup-scheduler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for warmup-scheduler: filename=warmup_scheduler-0.3-py3-none-any.whl size=2983 sha256=f5497449be9eeb0b1d0bc1e3e466c90db43eff9babdec6d3312339ac621c89a8
  Stored in directory: /root/.cache/pip/wheels/f2/ce/4a/215c4f0add432420ff90fe04656bf2664ddfac7302e2b6fe51
Successfully built warmup-scheduler


**4)** The .yml file includes the training hyperparameters (patch size, batch size, learning rate, etc.). Inaddition to those, the directories of the saved models and the datasets are given in the .yml file. We need to read this file here:

In [7]:
from config import Config 
opt = Config('training6.yml')

gpus = ','.join([str(i) for i in opt.GPU])
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = gpus

**5)** Import required libraries:

In [8]:
import torch
torch.backends.cudnn.benchmark = True

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

import random
import time
import numpy as np

import utils
from data_RGB import get_training_data, get_validation_data
from MPRNet import MPRNet
import losses
from warmup_scheduler import GradualWarmupScheduler
from tqdm import tqdm
from pdb import set_trace as stx

**6)** As mentioned, the PNSR calculation we use exploits random patches from the image. To make this calculation repetitive, we set the random seeds to fixed values to avoid changes each time we run the code:

In [9]:
######### Set Seeds ###########
random.seed(1234)
np.random.seed(1234)
torch.manual_seed(1234)
torch.cuda.manual_seed_all(1234)

start_epoch = 1
mode = opt.MODEL.MODE
session = opt.MODEL.SESSION

**7)** Find the validation and training datasets directories based on the given ".yml" file:

In [10]:
result_dir = os.path.join(opt.TRAINING.SAVE_DIR, mode, 'results', session)
model_dir  = os.path.join(opt.TRAINING.SAVE_DIR, mode, 'models',  session)

utils.mkdir(result_dir)
utils.mkdir(model_dir)

train_dir = opt.TRAINING.TRAIN_DIR
val_dir   = opt.TRAINING.VAL_DIR

**8)** Create dataloaders based on the dataset:

In [14]:
######### DataLoaders ###########
train_dataset = get_training_data(train_dir, {'patch_size':opt.TRAINING.TRAIN_PS})
train_loader = DataLoader(dataset=train_dataset, batch_size=opt.OPTIM.BATCH_SIZE, shuffle=True, num_workers=16, drop_last=False, pin_memory=True)

val_dataset = get_validation_data(val_dir, {'patch_size':opt.TRAINING.VAL_PS})
val_loader = DataLoader(dataset=val_dataset, batch_size=16, shuffle=False, num_workers=8, drop_last=False, pin_memory=True)

print('===> Start Epoch {} End Epoch {}'.format(start_epoch,opt.OPTIM.NUM_EPOCHS + 1))
print('===> Loading datasets')

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


===> Start Epoch 1 End Epoch 3001
===> Loading datasets


**9)** This function gets the dataloader and defined model, and calculates the PNSR:

In [11]:
def get_pnsr(data_loader, model):
    psnr_val_rgb = []
    for ii, data_val in enumerate((data_loader), 0):
        target = data_val[0].cuda()
        input_ = data_val[1].cuda()
        restored = input_

        if model:
            model.eval()
            with torch.no_grad():
                restored = model(input_)
            restored = restored[0]

        for res,tar in zip(restored,target):
            psnr_val_rgb.append(utils.torchPSNR(res, tar))

    psnr_val_rgb  = torch.stack(psnr_val_rgb).mean().item()

    return psnr_val_rgb

**10)** This function finds the model in the "name" directory and calls the previous function:

In [12]:
def eval_mprnet(name, data_loader):
  mprnet = MPRNet()
  mprnet.cuda()
  weights = utils.get_last_path(model_dir, name)

  # utils.load_checkpoint(mprnet,weights)
  # mprnet.load_state_dict(torch.load)

  # mprnet.load_state_dict(torch.load)

  checkpoint = torch.load(weights)
  mprnet.load_state_dict(checkpoint["state_dict"])
  return get_pnsr(data_loader, mprnet)

**11)** Results:

Our goal is to improve the quality of test dataset images by deblurring. The PNSR of test dataset is what we need to increase. It is important to have some sense about the PNSR of the dataset we are trying to deblur:

 

In [20]:
print("Test Dataset PNSR:", get_pnsr(val_loader, None))

Test Dataset PNSR: 26.814794540405273


We need to know how the ***pretrained model*** (our initial model) works on the dataset:

In [19]:
print("Pretrained Model PNSR Over Test Dataset:", eval_mprnet("_pretrained.pth", val_loader))

Pretrained Model PNSR Over Test Dataset: 27.67173194885254


The performance of the ***fine-tuned model*** on the dataset:

In [21]:
print("Fine-Tuned Model PNSR Over Test Dataset:", eval_mprnet("_best.pth", val_loader))

Fine-Tuned Model PNSR Over Test Dataset: 31.992860794067383
